# Using three.js with js_proxy

This notebook transcribes the example from the 
[three.js README](https://github.com/mrdoob/three.js)
into the equivalent incantations using the `js_proxy`
proxy widget mechanism.

This may not be the best way to use `three.js` inside
Jupyter/IPython notebooks.  I recommend taking a look at
[pythreejs](https://github.com/jovyan/pythreejs)
for a sophisticated and tuned embedding of `three.js` functionality
in a IPython widget implementation.

However, this proof of concept demonstrates that `js_proxy`
can be useful.

To start, load the `three.js` minified javascript from
a CDN location.

In [20]:
from IPython import display
load_three = """
<script src="http://cdnjs.cloudflare.com/ajax/libs/three.js/r70/three.min.js">
</script>
"""
display.display(display.HTML(load_three))

Then load the js_proxy Python and Javascript support and create a proxy widget
to use as a communication link to the browser/Javascript context.

Also add some convenience variables and functions.

In [21]:
from jp_gene_viz import js_proxy
js_proxy.load_javascript_support()

In [22]:
w = js_proxy.ProxyWidget()

# Some shortcut names for proxy references for convenience:
# The global window namespace:
window = w.window()
# The jQuery element for the widget:
element = w.element()
# The THREE module object:
THREE = window.THREE
# The "set" operation to store a JS value
save_js = element._set
# The emulation of the JS "new" keyword.
new = element.New

def save_new(name, constructor, arguments):
    """
    Create an object and save it in the element namespace,
    """
    new_reference = new(constructor, arguments)
    return save_js(name, new_reference)

Now we create "proxy commands" which emulate the actions of the `init` function of the README.

In [24]:
save_scene = save_new("scene", THREE.Scene, [])
save_camera = save_new("camera", THREE.PerspectiveCamera, [75, 1.0, 1, 10000])
set_position = element.camera.position._set("z", 1000)
save_geometry = save_new("geometry", THREE.BoxGeometry, [200, 200, 200])
save_material = save_new("material", THREE.MeshBasicMaterial, [{"color": 0xff0000, "wireframe": True } ])
save_mesh = save_new("mesh", THREE.Mesh, [element.geometry, element.material])
add_mesh = element.scene.add(element.mesh)
save_renderer = save_new("renderer", THREE.WebGLRenderer, [])
set_sizes = element.renderer.setSize(300, 300)
add_renderer = element.append(element.renderer.domElement)
do_render = element.renderer.render(element.scene, element.camera)
json_sent = w.send_commands([
    save_scene,
    save_camera,
    set_position,
    save_geometry,
    save_material,
    save_mesh,
    add_mesh,
    save_renderer,
    set_sizes,
    add_renderer,
    do_render,
])

In [25]:
# show the 3d scene.
display.display(w)

In [34]:
# rotate the cube using a busy-loop blocking the interpreter.
import time

def make_it_rotate():
    for i in xrange(500):
        time.sleep(0.1)
        rotate = element.mesh.rotation._set("x", i/10.0)._set("y", i/5.0)
        w.send_commands([rotate, do_render])

make_it_rotate()

In [35]:
# rotate the cube using a requestAnimationFrame callback
# this doesn't block the interpreter.
requestAnimationFrame = window.requestAnimationFrame
rotation = {"x": 1.1, "y": 2.2, "count": 0}

def animation_frame(data=None, arguments=None):
    rotation["count"] += 1
    if rotation["count"] > 100000:
        return # stop animation
    rotation["x"] += 0.01
    rotation["y"] += 0.02
    rotate = element.mesh.rotation._set("x", rotation["x"])._set("y", rotation["y"])
    next_frame = requestAnimationFrame(animation_callback)
    w.send_commands([rotate, do_render, next_frame])

# set up the js-->python callback interface
animation_callback = w.callback(animation_frame, data=None)

# start the animation sequence
animation_frame()

In [33]:
# do some calculation during the animation
12 + 90

102